# Building a simple search engine
## Part 2: Inverted Index

In this exercise, we will build a inverted search index, and then search the dataset using it.

### Loading the dataset

For this exercise we will be loading both files from the dataset and generate a named tuple also containing the descriptions. This may take roughly a minute depending on the computing power of your device.

In [1]:
import pickle, lzma
from collections import namedtuple

summaries_file = 'data/arxiv_cs_summaries.pickle.xz'
descriptions_file = 'data/arxiv_cs_descriptions.pickle.xz'

summaries = pickle.load(lzma.LZMAFile(summaries_file, 'rb'))
descriptions = pickle.load(lzma.LZMAFile(descriptions_file, 'rb'))

paper = namedtuple("paper", ["title", "authors", "year", "link", "description"])
papers = [paper(*summaries[i], descriptions[i]) for i in range(len(summaries))]

Let's take a look at an example paper:

In [2]:
print(papers[0])

paper(title='Intelligent location of simultaneously active acoustic emission sources:  Part I', authors=['Kosel, T.', 'Grabec, I.'], year=2007, link='http://arxiv.org/abs/0704.0047', description='The intelligent acoustic emission locator is described in Part I, while Part II discusses blind source separation, time delay estimation and location of two simultaneously active continuous acoustic emission sources.   The location of acoustic emission on complicated aircraft frame structures is a difficult problem of non-destructive testing. This article describes an intelligent acoustic emission source locator. The intelligent locator comprises a sensor antenna and a general regression neural network, which solves the location problem based on learning from examples. Locator performance was tested on different test specimens. Tests have shown that the accuracy of location depends on sound velocity and attenuation in the specimen, the dimensions of the tested area, and the properties of store

## Setup

The code below defines a function `print_paper(paper_id, print_description=False, highlight=[])` to print the summary of a paper (with descriptions) given its id, and optionally highlight parts of the description given a list of strings. You can use this to quickly test your search results.

In [3]:
from IPython.display import display, HTML

def print_paper(paper_id, print_description=False, highlight=[]):
    p = papers[paper_id]
    display(HTML(f"<b><a href='{p.link}'>{p.title}</a></b><br>{p.year} - {', '.join(p.authors)}"))
    if print_description:
        description = descriptions[paper_id]
        for h in highlight:
            description = description.replace(h, f"<span style='background: yellow'>{h}</span>")
        display(HTML(description))

In [4]:
for p in range(3):
    print_paper(p)

## Inverted index

What is the most basic problem a search engine has to solve? Given a search term as input, output a list of results containing the search term. More specifically, given a `string` as input, return a list of papers whose descriptions contain the `string`. So how do we do that?

"The answer is quite straightforward", you say, "just loop over the papers and if the paper contain the result, put it in a list, something like this":

```
def search(query):
    return [p for p in papers if query in p.description]
```

The function above would indeed return a list of papers with the results, but there are a few problems:
- This function only returns exact matches of your search query, i.e. `"Computer" != "computer" != "computers"`
- The search function takes a while to run (and will take even longer as the dataset becomes bigger)

The above reason (mostly the performance one) is why we would use an inverted index. First let's talk about the index we have learned about. We have a list of tuples, and each tuple contains the properties of a paper (title, authors, year, link, description). This list can be treated as an index: we can get one paper using its position (index) in the list.

An inverted index is created differently: we create an index based on the **terms** (similar to words) within the descriptions. Instead of getting one paper by its position (i.e. `papers[12345]` returns a `tuple`), we get a list of papers containing a term by the term itself (i.e. `index["computer"]` returns a `list` of papers containing the term `"computer"` in the description).

To create an inverted index, first we process the descriptions into a list of terms, and then for each of those terms we create a set of articles containing that word. If we rephrase it into Python terms, we are creating a `dict` whose keys are the terms (`string`) and the results are a `set` of indices (`int`) of the articles whose description contains that term, for example:
```
{
    "computer": {1, 2, 3, 4, 5},
    "network": {2, 5, 8, 9},
    "term": {1, 11},
    ...
}
```

### Preprocessing the text

Two of the common steps in producing a list of **terms** from a text: **tokenization** and **stemming**.

#### Tokenization
Tokenization breaks a body of text into individual semantic units called **tokens**, for example:
```
"The quick brown fox jumps over the lazy dog."
```
becomes
```
['The', 'quick', 'brown', 'fox', 'jumps', 'over', 'the', 'lazy', 'dog.']
```

#### Stemming
Stemming reduces/normalises tokens to a base *stem* so that we don't need an exact match for the same word in another form, for example *is*, *am*, *are* can be reduced to *be*, and in the above sentence *The* can be reduced to *the*.

To keep things simple, we will tokenize the paper descriptions by splitting the text by space, and then stem the tokens by converting them into lower case. Let's define the two functions below:

In [5]:
def tokenize(text):
    return text.split(" ")

def stem(tokens):
    return [t.lower() for t in tokens]

## Creating the index

In [6]:
from collections import defaultdict

# A defaultdict creates a set by defualt when we save items to a new key
inverted_index = defaultdict(set)

# This may take a while
for (i, p) in enumerate(papers):
    for term in stem(tokenize(p.description)):
        inverted_index[term].add(i)

## Querying the index
Now that we have created the index let's test it with some queries. The code below should return 38618 results:

In [7]:
query = "learning"

# display(HTML(f"<h3>There are {len(inverted_index[query])} results for '{query}':</h3><hr>"))

for i in list(inverted_index[query])[:5]:
    print_paper(i)

The inverted index we created above can be searched by a single word, but we rarely want to search with only one term. To find documents containing multiple search terms, we can look into the sets of papers for each search term and find the intersection of all of them.

**Exercise:** Define a function `and_query(query)` that returns a set of articles that match all input terms. The query for `machine learning` should return 7944 results.

*Hint:* Tokenize and stem your query first using `tokenize()` and `stem()`. The results in the inverted index are `set`s, which means you can perform set operations.

In [8]:
def and_query(query):
    terms = stem(tokenize(query))
    # Find the intersection of the terms and return it
    results = inverted_index[terms[0]]
    for term in terms[1:]:
        results = results.intersection(inverted_index[term])
    return results

In [9]:
example_query = "machine learning"
example_result = and_query(example_query)
display(HTML(f"<h3>There are {len(example_result)} results for '{example_query}':</h3><hr>"))
for i in list(example_result)[:5]:
    print_paper(i)

## Problems with Tokenization and Stemming

### Tokenization

The paper at index 35827 is related to neural networks, yet they are not part of the results when you look for `neural network`, why is that the case?

In [10]:
query = "neural network"
i = 35827

print(f"Is paper {i} in the search results for '{query}'?", i in and_query(query))
print_paper(i, True, stem(tokenize(query)))

Is paper 35827 in the search results for 'neural network'? False


You can see that the word `neural network` is highlighted in the description, after `network` there is a full stop. If you change the query to `neural network.`, this paper will be in the results. This demonstrates that tokenization isn't as simple as breaking a sentence by space. Here are some examples where tokenization becomes complicated:

- Den Haag
- Lebensversicherungsgesellschaftsangestellter (life insurance company employee)
- ziektekostenverzekeringsmaatschappij (health insurance company)
- Languages with no spaces (e.g. Chinese, Japanese)

### Stemming

As you can see below, the paper with id 36090 has the word `neural network` in the title, yet it is not in the results when we search for `neural network`. Why is it the case?

In [11]:
query = "neural network"
i = 36090

print(f"Is paper {i} in the search results for '{query}'?", i in and_query(query))
# print(descriptions[i])
print_paper(i, True, stem(tokenize(query)))

Is paper 36090 in the search results for 'neural network'? False


The description only has `neural networks` but not `neural network`, and our stemming function does not reduce `networks` to `network`. This paper would show up if you look for `neural networks`. But how do we stem all the words that can appear in the language?

## Improved inverted index with `nltk`

Luckily, there is a module [`nltk`](http://www.nltk.org/) in Python that simplifies all those challenges we see above. `nltk` should be installed when you install Anaconda, if not you can run this in a terminal to install it: `conda install -c anaconda nltk`

Let's define `tokenize2()` and `stem2()` to better process the dataset:

In [12]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import EnglishStemmer
nltk.download('punkt')     # Download the pre-trained Punkt tokenizer
stemmer = EnglishStemmer() # Initialise the English stemmer

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\yoyo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [13]:
# word_tokenize treats punctuation as tokens
# So we check the tokens with str.isalnum() to see if they are alphanumeric
# Obviously this only works with English
def tokenize2(text):
    return [t for t in set(word_tokenize(text)) if t.isalnum()]

def stem2(tokens):
    return [stemmer.stem(t) for t in tokens]

sentence = "machine learning"
print(stem(tokenize(sentence)))
print(stem2(tokenize2(sentence)))

['machine', 'learning']
['machin', 'learn']


Now we will rebuild the index using the NLTK-powered functions (only a subset of 5000 papers are used since these functions are a lot slower):

**Exercise:** Build another inverted index using the subset of articles and the NLTK-enhanced functions defined above and save it in `smarter_index`

In [14]:
smarter_index = defaultdict(set)

subset = papers[:5000]

# This may take a while
for (i, p) in enumerate(subset):
    for term in stem2(tokenize2(p.description)):
        smarter_index[term].add(i)

Just for comparison, let's rebuild the naïve inverted index using the same subset:

In [15]:
inverted_index = defaultdict(set)

for (i, p) in enumerate(subset):
    for term in stem(tokenize(p.description)):
        inverted_index[term].add(i)

Let's look at the number of keys both indexes have:

In [16]:
print(len(inverted_index.keys()))
print(len(smarter_index.keys()))

41897
11874


**Question:** Explain the difference in the number of keys and search results.

**Answer:** The newer tokenization and stemming functions are better at splitting sentences and grouping the tokens, resulting in fewer keys and more results under a term.

**Exercise:** Create a new query function `and_query2(query)` using the new tokenization and stemming functions and the smarter inverted index.

In [17]:
def and_query2(query):
    terms = stem2(tokenize2(query))
    # Find the intersection of the terms and return it
    results = smarter_index[terms[0]]
    for term in terms[1:]:
        results = results.intersection(smarter_index[term])
    return results

Let's compare the search results of the two indices:

In [18]:
example_query = "The Who"
example_result = and_query(example_query)
example_result2 = and_query2(example_query)
print(f"There are {len(example_result)} results for '{example_query}' using the naïve index")
print(f"There are {len(example_result2)} results for '{example_query}' using the smarter index")

There are 72 results for 'The Who' using the naïve index
There are 80 results for 'The Who' using the smarter index


**Question:** Given the nature of our dataset, how many of the papers from above do you think are actually about [The Who](http://en.wikipedia.org/wiki/The_Who)? What could you do to prevent these kind of incorrect results?

**Answer:** Probably none of the matches is really about the band. Because we are searching the terms individually and not as a phrase and because 'the' and 'who' and both very common words in English, we get many results where the two occur, but probably not after each other (and even then probably not referring to the band).

## Conclusion

In this exercise, we looked into how we can search a library of texts using an inverted index, and how we can improve the process of building the index to get more relevant results. However, this is just the beginning, there are a lot more improvements we can do. For now all results have the same "relevance", how do we create a ranking out of all these results? (hint: [tf-idf](http://www.tfidf.com/), [PageRank](https://www.cs.princeton.edu/~chazelle/courses/BIB/pagerank.htm)) How do we know the context of a search, or the relations between objects in real life? (Knowledge graph [[1]](https://googleblog.blogspot.com/2012/05/introducing-knowledge-graph-things-not.html), [[2]](http://ceur-ws.org/Vol-1695/paper4.pdf)) [There](https://en.wikipedia.org/wiki/Information_retrieval) [are](https://arxiv.org/list/cs.IR/recent) [fields](https://en.wikipedia.org/wiki/Natural_language_processing) dedicated to extracting information from data, and if you want to know more, [this book](https://nlp.stanford.edu/IR-book/) is a good start.

All theories aside, I was hoping the exercises give you more confidence with Python, since a search engine is nothing more than a dict of terms and matching results, and the harder part is usually with understanding the problem (in this case information retrieval theory).